In [1]:
#importation
import numpy as np
import pandas as pd
from textblob import TextBlob 
import re
import nltk
from pandas import DataFrame
from nrclex import NRCLex
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [2]:
filePath='C:/Users/pk/Desktop/projet/analyse/data/samsungtweet.csv'
data=pd.read_csv(filePath)
data.columns=["date", "text"]
data.head(5)

,date,text
0,2021-03-19 23:59:44,b'@TrethanSW And it doesn\xe2\x80\x99t have al...
1,2021-03-19 23:59:31,b'SAMSUNG GALAXY A52 UNBOXING and GIVEAWAY\xf0...
2,2021-03-19 23:59:28,"b'Samsung Galaxy A51 5G, Galaxy A71 5G Get And..."
3,2021-03-19 23:58:54,"b""RT @FromKorea5: Samsung Galaxy Z Fold 2 Whit..."
4,2021-03-19 23:58:51,"b""RT @WhitestoneChina: \xe2\x9d\x94\xe2\x9d\x9..."


In [3]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,date,0.0
1,text,0.0


In [4]:
#**Drop NAN Values**
data =data.dropna()

In [5]:
#**Verify NAN Values**
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,date,0.0
1,text,0.0


In [6]:
#**Convert the data into lowercase**
data["Comment"]= data["text"].str.lower()

In [7]:
#**Word Tokenization and deleting punctuation**
AComment=[]
for comment in data["Comment"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    AComment.append(Word_Tok)

In [8]:
#**Add New Column to our data**
data["Word_Tok"]= AComment
data.head()

,date,text,Comment,Word_Tok
0,2021-03-19 23:59:44,b'@TrethanSW And it doesn\xe2\x80\x99t have al...,b'@trethansw and it doesn\xe2\x80\x99t have al...,"[b, trethansw, and, it, doesn, xe2, x80, x99t,..."
1,2021-03-19 23:59:31,b'SAMSUNG GALAXY A52 UNBOXING and GIVEAWAY\xf0...,b'samsung galaxy a52 unboxing and giveaway\xf0...,"[b, samsung, galaxy, a52, unboxing, and, givea..."
2,2021-03-19 23:59:28,"b'Samsung Galaxy A51 5G, Galaxy A71 5G Get And...","b'samsung galaxy a51 5g, galaxy a71 5g get and...","[b, samsung, galaxy, a51, 5g, galaxy, a71, 5g,..."
3,2021-03-19 23:58:54,"b""RT @FromKorea5: Samsung Galaxy Z Fold 2 Whit...","b""rt @fromkorea5: samsung galaxy z fold 2 whit...","[b, rt, fromkorea5, samsung, galaxy, z, fold, ..."
4,2021-03-19 23:58:51,"b""RT @WhitestoneChina: \xe2\x9d\x94\xe2\x9d\x9...","b""rt @whitestonechina: \xe2\x9d\x94\xe2\x9d\x9...","[b, rt, whitestonechina, xe2, x9d, x94, xe2, x..."


In [9]:
#**Delete StopWords**
        
description_list =[]
for description in data['Word_Tok']:
   
    description=[word for word in description if not word in set(stopwords.words("english"))]
    lemma=nltk.WordNetLemmatizer()
    description=[lemma.lemmatize(word) for word in description]
    description=" ".join(description)
    description_list.append(description)


In [10]:
#**Add new Column: Comment after preprocessing**

data["CommentAferPreproc"]=description_list
data.head()

,date,text,Comment,Word_Tok,CommentAferPreproc
0,2021-03-19 23:59:44,b'@TrethanSW And it doesn\xe2\x80\x99t have al...,b'@trethansw and it doesn\xe2\x80\x99t have al...,"[b, trethansw, and, it, doesn, xe2, x80, x99t,...",b trethansw xe2 x80 x99t bloat ware company ad...
1,2021-03-19 23:59:31,b'SAMSUNG GALAXY A52 UNBOXING and GIVEAWAY\xf0...,b'samsung galaxy a52 unboxing and giveaway\xf0...,"[b, samsung, galaxy, a52, unboxing, and, givea...",b samsung galaxy a52 unboxing giveaway xf0 x9f...
2,2021-03-19 23:59:28,"b'Samsung Galaxy A51 5G, Galaxy A71 5G Get And...","b'samsung galaxy a51 5g, galaxy a71 5g get and...","[b, samsung, galaxy, a51, 5g, galaxy, a71, 5g,...",b samsung galaxy a51 5g galaxy a71 5g get andr...
3,2021-03-19 23:58:54,"b""RT @FromKorea5: Samsung Galaxy Z Fold 2 Whit...","b""rt @fromkorea5: samsung galaxy z fold 2 whit...","[b, rt, fromkorea5, samsung, galaxy, z, fold, ...",b rt fromkorea5 samsung galaxy z fold 2 whites...
4,2021-03-19 23:58:51,"b""RT @WhitestoneChina: \xe2\x9d\x94\xe2\x9d\x9...","b""rt @whitestonechina: \xe2\x9d\x94\xe2\x9d\x9...","[b, rt, whitestonechina, xe2, x9d, x94, xe2, x...",b rt whitestonechina xe2 x9d x94 xe2 x9d x94 x...


In [11]:
#**Sentiment Analysis with TextBlob**

senti_list = []
for i in data["CommentAferPreproc"]:
   # vs=te.get_emotion(i)
    vs = TextBlob(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')  


In [12]:
#**Add Column: Sentiment**
data["sentiment"]=senti_list
data.head()


,date,text,Comment,Word_Tok,CommentAferPreproc,sentiment
0,2021-03-19 23:59:44,b'@TrethanSW And it doesn\xe2\x80\x99t have al...,b'@trethansw and it doesn\xe2\x80\x99t have al...,"[b, trethansw, and, it, doesn, xe2, x80, x99t,...",b trethansw xe2 x80 x99t bloat ware company ad...,Negative
1,2021-03-19 23:59:31,b'SAMSUNG GALAXY A52 UNBOXING and GIVEAWAY\xf0...,b'samsung galaxy a52 unboxing and giveaway\xf0...,"[b, samsung, galaxy, a52, unboxing, and, givea...",b samsung galaxy a52 unboxing giveaway xf0 x9f...,Positive
2,2021-03-19 23:59:28,"b'Samsung Galaxy A51 5G, Galaxy A71 5G Get And...","b'samsung galaxy a51 5g, galaxy a71 5g get and...","[b, samsung, galaxy, a51, 5g, galaxy, a71, 5g,...",b samsung galaxy a51 5g galaxy a71 5g get andr...,Neutral
3,2021-03-19 23:58:54,"b""RT @FromKorea5: Samsung Galaxy Z Fold 2 Whit...","b""rt @fromkorea5: samsung galaxy z fold 2 whit...","[b, rt, fromkorea5, samsung, galaxy, z, fold, ...",b rt fromkorea5 samsung galaxy z fold 2 whites...,Positive
4,2021-03-19 23:58:51,"b""RT @WhitestoneChina: \xe2\x9d\x94\xe2\x9d\x9...","b""rt @whitestonechina: \xe2\x9d\x94\xe2\x9d\x9...","[b, rt, whitestonechina, xe2, x9d, x94, xe2, x...",b rt whitestonechina xe2 x9d x94 xe2 x9d x94 x...,Neutral


In [13]:
data.to_csv("C:/Users/pk/Desktop/projet/analyse/out_data/TweetAfterProcess.csv")



In [14]:
text=" ".join(data['CommentAferPreproc'])
text_object = NRCLex(text)

In [15]:
sentiment_scores = pd.DataFrame(list(text_object.raw_emotion_scores.items()))
sentiment_scores = sentiment_scores.rename(columns={0: "Sentiment", 1: "Count"})
sentiment_scores

#sentiment_scores.to_csv('C:/Users/pk/Desktop/projet/analyse/out_data/tweetSentimentCount.csv')

,Sentiment,Count
0,fear,257
1,negative,342
2,anger,205
3,disgust,143
4,sadness,184
5,positive,910
6,trust,514
7,anticipation,424
8,joy,221
9,surprise,187


In [16]:
sentiment_words = pd.DataFrame(list(text_object.affect_dict.items()),columns = ['words','sentiments'])
sentiment_words
#sentiment_words.to_csv('C:/Users/pk/Desktop/projet/analyse/out_data/tweetWordSentiment.csv')


,words,sentiments
0,ware,"[fear, negative]"
1,hate,"[anger, disgust, fear, negative, sadness]"
2,protector,"[positive, trust]"
3,watch,"[anticipation, fear]"
4,time,[anticipation]
...,...,...
426,fawn,[negative]
427,nerve,[positive]
428,serene,"[negative, trust]"
429,fore,[positive]


In [17]:
sentiment = sentiment_scores['Sentiment'].to_list()
sentiment

['fear',
 'negative',
 'anger',
 'disgust',
 'sadness',
 'positive',
 'trust',
 'anticipation',
 'joy',
 'surprise']

In [18]:
for y in sentiment:
    sentiment_words[y] = 9
sentiment_words

,words,sentiments,fear,negative,anger,disgust,sadness,positive,trust,anticipation,joy,surprise
0,ware,"[fear, negative]",9,9,9,9,9,9,9,9,9,9
1,hate,"[anger, disgust, fear, negative, sadness]",9,9,9,9,9,9,9,9,9,9
2,protector,"[positive, trust]",9,9,9,9,9,9,9,9,9,9
3,watch,"[anticipation, fear]",9,9,9,9,9,9,9,9,9,9
4,time,[anticipation],9,9,9,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
426,fawn,[negative],9,9,9,9,9,9,9,9,9,9
427,nerve,[positive],9,9,9,9,9,9,9,9,9,9
428,serene,"[negative, trust]",9,9,9,9,9,9,9,9,9,9
429,fore,[positive],9,9,9,9,9,9,9,9,9,9


In [19]:
a=0
for i in sentiment_words['sentiments']: 
    for y in sentiment:
        sentiment_words[y][a] = int(y in i)
    a=a+1

C:\Users\pk\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
sentiment_words.head(5)

,words,sentiments,fear,negative,anger,disgust,sadness,positive,trust,anticipation,joy,surprise
0,ware,"[fear, negative]",1,1,0,0,0,0,0,0,0,0
1,hate,"[anger, disgust, fear, negative, sadness]",1,1,1,1,1,0,0,0,0,0
2,protector,"[positive, trust]",0,0,0,0,0,1,1,0,0,0
3,watch,"[anticipation, fear]",1,0,0,0,0,0,0,1,0,0
4,time,[anticipation],0,0,0,0,0,0,0,1,0,0


In [21]:
for y in sentiment:
    word_list = sentiment_words[sentiment_words[y]== 1].words.head(10)
    print(f"Sentiment:{y}")
    print(f"{word_list.values}\n")

Sentiment:fear
['ware' 'hate' 'watch' 'lightning' 'disruption' 'shortage' 'surgery'
 'bang' 'ruined' 'hearing']

Sentiment:negative
['ware' 'hate' 'dust' 'weird' 'storm' 'disruption' 'shortage' 'detention'
 'bang' 'ruined']

Sentiment:anger
['hate' 'lightning' 'storm' 'disruption' 'shortage' 'wireless' 'bang'
 'ruined' 'penalty' 'shot']

Sentiment:disgust
['hate' 'weird' 'finally' 'bang' 'ruined' 'gray' 'horrible' 'hell' 'ill'
 'winning']

Sentiment:sadness
['hate' 'shortage' 'surgery' 'detention' 'bang' 'ruined' 'penalty' 'black'
 'shot' 'gray']

Sentiment:positive
['protector' 'protect' 'tempered' 'impression' 'visit' 'promise' 'provide'
 'reliable' 'safe' 'production']

Sentiment:trust
['protector' 'series' 'promise' 'provide' 'reliable' 'safe' 'finally'
 'theory' 'justice' 'appreciation']

Sentiment:anticipation
['watch' 'time' 'edition' 'production' 'wireless' 'finally' 'inquiry'
 'mobile' 'theory' 'start']

Sentiment:joy
['promise' 'safe' 'finally' 'appreciation' 'dance' 'pay' 'd